### Install Library

In [ ]:
!pip install miditoolkit -q --use-deprecated=legacy-resolver
!pip install transformers[torch] -q --use-deprecated=legacy-resolver
!pip install bitsandbytes -q --use-deprecated=legacy-resolver
!pip install datasets -q --use-deprecated=legacy-resolver
!pip install peft -q --use-deprecated=legacy-resolver
!pip install datasets
!pip install rouge_score


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dataa1/data1.csv
/kaggle/input/fullda/full_data.csv
/kaggle/input/hihihaha/data1.csv


In [ ]:
def load_vocab(vocab_dir):
  lines = []
  lines = ""
  with open(vocab_dir, "r") as f:
    lines = f.readlines()
  vocab = []
  for ele in lines:
    vocab.append(ele.split(" ")[0])
  return vocab

In [ ]:
!pip install gdown
!gdown --id 1vIRxkZp8TUz_cMGMuBqBNBcIFg0GsA68

In [ ]:
!gdown --id 18Ci6oaeoARqpkHadNv6y8_KW9UmW8hlr

In [ ]:
# !unzip /kaggle/working/checkpoint-5988.zip

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/tokenizer101.json")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


0

In [ ]:
import transformers
from datasets import load_dataset, load_metric

# Tải dataset từ file CSV
medium_datasets = load_dataset("csv", data_files="/data.csv")

# Chia dữ liệu thành train, validation và test
datasets_train_test = medium_datasets["train"].train_test_split(test_size=0.05)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=0.1)

medium_datasets = {
    "train": datasets_train_validation["train"],
    "validation": datasets_train_validation["test"],
    "test": datasets_train_test["test"]
}

context_length = 125
music_max_length = 1024  

# Function for tokenizing the dataset
def tokenize_function(examples):
    input_ids = []
    attention_masks = []
    labels = []

    for text in examples['0']:
        split_data = text.split('<sep>')
        if len(split_data) != 2:
            continue
        prompt = split_data[0].strip()
        music = split_data[1].strip()

        combined_text = prompt + " " + music

        tokenized = tokenizer(combined_text, padding="max_length", truncation=True, max_length=context_length + music_max_length)

        labels_combined = tokenized['input_ids'].copy()

        labels_combined = [-100 if token == tokenizer.pad_token_id else token for token in labels_combined]

        input_ids.append(tokenized['input_ids'])
        attention_masks.append(tokenized['attention_mask'])
        labels.append(labels_combined)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

# Áp dụng hàm tokenize_function cho từng phần dataset
tokenized_datasets = {}
for split in ["train", "validation", "test"]:
    tokenized_datasets[split] = medium_datasets[split].map(
        tokenize_function, batched=True, remove_columns=medium_datasets[split].column_names
    )


Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
medium_datasets["train"][0]

In [ ]:
n_layer=20# Number of transformer layers
n_head=16 # Number of multi-head attention heads
n_emb=1024 # Embedding size

from transformers import AutoConfig, GPT2LMHeadModel

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_positions=2048,
    n_layer=n_layer,
    n_head=n_head,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_embd=n_emb)

model = GPT2LMHeadModel(config)
model.resize_token_embeddings(len(tokenizer))
model.tokenizer = tokenizer

,

In [ ]:
model.eval()

In [ ]:
from peft import LoraConfig

# lora_config = LoraConfig( r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type= "CAUSAL_LM",target_modules=["q", "v"] )
#target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head',"fc_in", "fc_out", "wte"]
target_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out", "wte", "lm_head"]

lora_config = LoraConfig(
r=16,
lora_alpha=12,
lora_dropout=0.1,
bias="none",
target_modules = target_modules,
task_type="CAUSAL_LM")
from peft import get_peft_model, LoraConfig, TaskType


base_model_with_new_adapter = get_peft_model(_model, lora_config)
base_model_with_new_adapter.print_trainable_parameters()

trainable params: 72,928 || all params: 255,381,728 || trainable%: 0.0286


In [ ]:

from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_metric
from argparse import Namespace

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Configuration parameters
config = {
    "output_dir": "output_29_6",
    "num_train_epochs": 10,
    "fp16": True,
    "optim": "paged_adamw_8bit",
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "evaluation_strategy": "steps",
    "save_strategy": "steps",
    "eval_steps": 500,
    "logging_steps": 500,
    "logging_first_step": True,
    "save_total_limit": 2,
    "save_steps": 500,
    "lr_scheduler_type": "cosine",
    "learning_rate": 5e-4,
    "warmup_steps": 5000,
    "weight_decay": 0.01,
    "seed": 1,
    "max_steps": 50000,
    "gradient_accumulation_steps": 2
}
args = Namespace(**config)
train_args = TrainingArguments(**config)

# Load the ROUGE metric
rouge_metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids

    # Convert predictions and labels from tensors to lists of integers
    predictions = predictions.tolist()
    labels = labels.tolist()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result

# Initialize the Trainer
trainer = Trainer(
    model=base_model_with_new_adapter,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

# Train the model
#trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=

In [ ]:
s = "I1s2_0 I1s2_1_2 I1s2_2_2 I1s2_3_2 I1s2_4_2 I1s2_5_2 I1s2_6_2 I1s2_7_2 I1s2_8_2 I1s2_9_2 I1s2_10_2 I1s2_11_2 I1s2_12_2 I1s2_13_2 I1s2_14_2 I1s2_15_2 I1s2_16_2 I1s2_17_2 I1s2_18_2 I1s2_19_2 I1s2_20_1 I1s2_21_2 I1s2_22_2 I1s2_23_2 I1s2_24_2 I1s2_25_2 I1s2_26_2 I1s2_27_2 I4_28 C1_4 R1_2 R3_3 S2s1_17 S4_0_2 S4_1_2 S4_2_2 S4_3_2 S4_4_2 S4_5_2 S4_6_2 S4_7_2 S4_8_2 S4_9_2 S4_10_2 S4_11_2 S4_12_2 S4_13_2 S4_14_2 S4_15_2 S4_16_2 S4_17_2 S4_18_2 S4_19_2 S4_20_2 S4_21_2 B1s1_4 TS1s1_7 K1_2 T1s1_3 P4_4 ST1_14 EM1_4 TM1_5 <sep> S4_14_2 S4_15_1 S4_16_2 S4_17_1 S4_18_2 S4_19_1 S4_20_1 S4_21_2 B1s1_3 TS1s1_0 K1_2 T1s1_1 P4_3 ST1_14 S4_3_2 S4_4_1 S4_5_1 S4_6_1 S4_7_1 S4_8_1 S4_9_1 S4_10_1 S4_11_1 S4_12_1 S4_13_2 S4_14_1 S4_15_2 S4_16_1 S4_17_1 S4_18_1 S4_19_1 S4_20_2 S4_21_1 B1s1_4 TS1s1_2 K1_2 T1s1_2 P4_5 ST1_14 I1s2_15_0 I1s2_16_1 I1s2_17_2 I1s2_18_1 I1s2_19_2 I1s2_20_2 I1s2_21_2 I1s2_22_1 I1s2_23_2 I1s2_24_1 I1s2_25_1 I1s2_26_2 I1s2_27_1 I4_28 S4_0_2 S4_1_1 S4_2_1 S4_3_1 S4_4_1 S4_5_2 I1s2_8_0".split(" ")
len(s)

In [ ]:
output = trainer.train()

In [ ]:
output = trainer.train(resume_from_checkpoint = True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
38000,5.216700,4.683202
38500,4.246200,4.459696
39000,3.989200,4.225923
39500,3.727400,3.950894
40000,3.436200,3.682172
40500,3.186500,3.486965
41000,3.024300,3.362039
41500,2.908100,3.280823
42000,2.822800,3.216054
42500,2.758900,3.173101


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

In [ ]:
save_directory = "/kaggle/working/custom_gpt2_model_28_6"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
from transformers import GPT2LMHeadModel, AutoTokenizer
save_directory = "/kaggle/working/custom_gpt2_model_28_6"

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = GPT2LMHeadModel.from_pretrained(save_directory,    device_map={"": "cpu"})


In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import GPT2LMHeadModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = GPT2LMHeadModel.from_pretrained(save_directory,    device_map={"": "cpu"})

_model = PeftModel.from_pretrained(model, "./output_28_6/checkpoint-45000", device_map={"": "cpu"})

In [ ]:
import torch
def generate_music(prompt_text, max_length=512):
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to("cuda")

    # Generate output
    with torch.no_grad():
        output_ids = trainer.model.generate(
            input_ids,
            max_length=max_length,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    # Decode output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text
prompt_text = "I1s2_0_0 I1s2_1_2 I1s2_2_2 I1s2_3_2 I1s2_4_2 I1s2_5_2 I1s2_6_2 I1s2_7_2 I1s2_8_2 I1s2_9_2 I1s2_10_2 I1s2_11_2 I1s2_12_2 I1s2_13_2 I1s2_14_2 I1s2_15_2 I1s2_16_2 I1s2_17_2 I1s2_18_2 I1s2_19_2 I1s2_20_1 I1s2_21_2 I1s2_22_2 I1s2_23_2 I1s2_24_2 I1s2_25_2 I1s2_26_2 I1s2_27_2 I4_28 C1_4 R1_2 R3_3 S2s1_17 S4_0_2 S4_1_2 S4_2_2 S4_3_2 S4_4_2 S4_5_2 S4_6_2 S4_7_2 S4_8_2 S4_9_2 S4_10_2 S4_11_2 S4_12_2 S4_13_2 S4_14_2 S4_15_2 S4_16_2 S4_17_2 S4_18_2 S4_19_2 S4_20_2 S4_21_2 B1s1_4 TS1s1_7 K1_2 T1s1_3 P4_4 ST1_14 EM1_4 TM1_5 <sep>"
generated_music = generate_music(prompt_text)
print("Generated Music:")
print(generated_music)

In [ ]:
prompt =  "I1s2_0_0 I1s2_1_2 I1s2_2_2 I1s2_3_2 I1s2_4_2 I1s2_5_2 I1s2_6_2 I1s2_7_2 I1s2_8_2 I1s2_9_2 I1s2_10_2 I1s2_11_2 I1s2_12_2 I1s2_13_2 I1s2_14_2 I1s2_15_2 I1s2_16_2 I1s2_17_2 I1s2_18_2 I1s2_19_2 I1s2_20_1 I1s2_21_2 I1s2_22_2 I1s2_23_2 I1s2_24_2 I1s2_25_2 I1s2_26_2 I1s2_27_2 I4_28 C1_4 R1_2 R3_3 S2s1_17 S4_0_2 S4_1_2 S4_2_2 S4_3_2 S4_4_2 S4_5_2 S4_6_2 S4_7_2 S4_8_2 S4_9_2 S4_10_2 S4_11_2 S4_12_2 S4_13_2 S4_14_2 S4_15_2 S4_16_2 S4_17_2 S4_18_2 S4_19_2 S4_20_2 S4_21_2 B1s1_4 TS1s1_7 K1_2 T1s1_3 P4_4 ST1_14 EM1_4 TM1_5"
#model1 = trainer.model
_model = _model.to("cuda")
input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
# Generate more tokens.
voice1_generated_ids = _model.generate(
    input_ids,
    max_length=512,
    do_sample=True,
    temperature=0.2,
    num_beams = 4,
    eos_token_id=tokenizer.encode("</s>")[0]
)
voice2_generated_ids = _model.generate(
    input_ids,
    max_length=512,
    do_sample=True,
    temperature=0.2,
    eos_token_id=tokenizer.encode("</s>")[0]
)
voice3_generated_ids = _model.generate(
    input_ids,
    max_length=512,
    do_sample=True,
    temperature=0.2,
    eos_token_id=tokenizer.encode("</s>")[0]
)
voice4_generated_ids = _model.generate(
    input_ids,
    max_length=512,
    do_sample=True,
    temperature=0.2,
    eos_token_id=tokenizer.encode("</s>")[0]
)
token_sequence = tokenizer.decode(voice4_generated_ids[0])

In [ ]:
# token_sequence = tokenizer.decode(voice3_generated_ids[0])
token_sequence = tokenizer.decode(voice2_generated_ids[0])

In [ ]:
token_sequence

In [ ]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    """
    Compresses the contents of a folder into a zip file.

    :param folder_path: Path to the folder to be compressed.
    :param zip_path: Path to the resulting zip file.
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage:
folder_to_zip = "/kaggle/working/custom_gpt2_model_ver3"
output_zip_file = "/kaggle/working/custom_gpt2_model_ver3.zip"

zip_folder(folder_to_zip, output_zip_file)


In [ ]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    """
    Compresses the contents of a folder into a zip file.

    :param folder_path: Path to the folder to be compressed.
    :param zip_path: Path to the resulting zip file.
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage:
folder_to_zip = "/kaggle/working/output_9_6"
output_zip_file = "/kaggle/working/output_9_6.zip"

zip_folder(folder_to_zip, output_zip_file)
